# Constant Force of Mortality

If force of mortality is constant, then future lifetime is exponentially distributed.  The exponential distribution is easy to work with, and has the _memoryless_ property that survival probability is independent of age (which is clearly an unrealistic assumption for human mortality).

$_tp_x = e^{-\mu t}$

$_tq_x = 1 - e^{-\mu t}$


- survival functions do not depend on age x; given any desired probability of survival (or death), the time-of-death threshold can also be easily derived by inverting this simple shortcut formula

## Expected future lifetime

$\overset{\circ}{e}_{x} = \dfrac{1}{\mu}$

- expected future lifetime is the mean of an exponential distribution, and does not depend on age x

$\overset{\circ}{e}_{x:\overline{n|}} = \dfrac{1}{\mu} (1 - e^{-\mu n})$

- temporary life expectancy is expected future life time, minus the probability of surviving $n$ years times the expected remaining lifetime of those survivors


$Var(T_x) = \dfrac{1}{\mu^2}$

- variance of future lifetime is the variance of an exponential distribution

## Pure endowment

$_nE_x = e^{-(\mu + \delta)n}$

- shortcut for pure endowment after substituting in the formula for survival function



## Life insurance

$\bar{A}_{x} = \dfrac{\mu}{\mu + \delta}$

- continuous whole life insurance, does not depend on age $x$

$~_{u|}\bar{A}_{x} = \dfrac{\mu}{\mu + \delta} e^{-\mu u}$

- deferred continuous whole life insurance as pure endowment times whole life insurance

$\bar{A}_{x:\overline{t|}} = \dfrac{\mu}{\mu + \delta} (1 - e^{-\mu t})$

- continuous term life insurance as difference of whole life and deferred whole life insurance shortcut formulas

## Life annuities

$\bar{a}_{x} = \dfrac{1}{\mu + \delta}$

- whole life annuity, does not depend on age $x$

$\bar{a}_{x:\overline{t|}} = \dfrac{1}{\mu + \delta} (1 - e^{-\mu t})$

- temporary life annuity as difference of whole life and deferred whole life annuity shortcut formulas

## Net future loss

$var(_0L) = \dfrac{\mu}{2\delta + \mu}$

- variance of loss at issue on whole life insurance with benefit premium calculated using the equivalence principle equals the second moment of insurance 


## Methods

The `ConstantForce` class specifies a constant force of mortality for the survival model. It inherits all the general methods for computing life contingency risks, and overrides those methods with shortcut formulas that are available when assuming an exponential distribution for future lifetime.

In [2]:

from actuarialmath import ConstantForce
import describe
describe.methods(ConstantForce)


class ConstantForce - Constant force of mortality - memoryless exponential distribution of lifetime

    Args:
      mu : constant value of force of mortality
      udd : assume UDD (True) or CFM (False, default) between integer ages

    Examples:
      >>> life = ConstantForce(mu=0.01).set_interest(delta=0.05)
      >>> life.term_insurance(35, t=35, discrete=False) + life.E_x(35, t=35)*0.51791

    Methods:
    --------

    e_x(x, s, t, curtate, moment):
      Expected lifetime E[T_x] is memoryless: does not depend on (x)

    E_x(x, s, t, endowment, moment):
      Shortcut for pure endowment: does not depend on age x

    whole_life_insurance(x, s, moment, b, discrete):
      Shortcut for APV of whole life: does not depend on age x

    temporary_annuity(x, s, t, b, variance, discrete):
      Shortcut for temporary life annuity: does not depend on age x

    term_insurance(x, s, t, b, moment, discrete):
      Shortcut for APV of term life: does not depend on age x

    Z_t(x, prob

## Examples


To use the constant force of mortality assumption, the `ConstantForce` subclass can be initialized with the value of `mu`; then shortcut formulas for computing life contingencies, where available, would be called instead of general inherited methods.

In [9]:
life = ConstantForce(mu=0.01).set_interest(delta=0.05)                       
life.term_insurance(35, t=35, discrete=False), life.E_x(35, t=35)   

(0.14625726195783623, 0.12245642825298157)

__SOA Question 6.36__

For a fully continuous 20-year term insurance policy of 100,000 on (50), you are given:
- Gross premiums, calculated using the equivalence principle, are payable at an annual rate of 4500
- Expenses at an annual rate of R are payable continuously throughout the life of the policy
- $\mu_{50 + t} = 0.04$, for $t > 0$
- $\delta = 0.08$

Calculate R.

In [4]:
print("SOA Question 6.36:  (B) 500")
life = ConstantForce(mu=0.04).set_interest(delta=0.08)
a = life.temporary_annuity(50, t=20, discrete=False)
A = life.term_insurance(50, t=20, discrete=False)
def fun(R):
    return life.gross_premium(a=a, A=A, initial_premium=R/4500,
                              renewal_premium=R/4500, benefit=100000)
R = life.solve(fun, target=4500, grid=[400, 800])
print(R)


SOA Question 6.36:  (B) 500
500.0


__SOA Question 6.31__

For a fully continuous whole life insurance policy of 100,000 on (35), you are given:
- The density function of the future lifetime of a newborn:
$$\begin{align*}
f(t) & = 0.01 e^{-0.01 t}, \quad 0 \le t < 70\\
& = g(t), \quad t \ge 70
\end{align*}$$

- $\delta = 0.05$
- $\overline{A}_{70} = 0.51791$

Calculate the annual net premium rate for this policy.

In [5]:
print("SOA Question 6.31:  (D) 1330")
life = ConstantForce(mu=0.01).set_interest(delta=0.05)
A = life.term_insurance(35, t=35) + life.E_x(35, t=35) * 0.51791 # A_35           
A = (life.term_insurance(35, t=35, discrete=False)
        + life.E_x(35, t=35) * 0.51791)    # A_35                                 
P = life.premium_equivalence(A=A, b=100000, discrete=False)
print(P)


SOA Question 6.31:  (D) 1330
1326.5406293909457


__SOA Question 6.27__

For a special fully continuous whole life insurance on (x), you are given:
- Premiums and benefits:

| | First 20 years | After 20 years |
| --- | --- | --- |
| Premium Rate | 3P | P |
| Benefit | 1,000,000 | 500,000 |


- $\mu_{x+t} = 0.03, \quad t \ge 0$
- $\delta = 0.06$

Calculate $P$ using the equivalence principle.

In [6]:
print("SOA Question 6.27:  (D) 10310")
life = ConstantForce(mu=0.03).set_interest(delta=0.06)
x = 0
payments = (3 * life.temporary_annuity(x, t=20, discrete=False)
            + life.deferred_annuity(x, u=20, discrete=False))
benefits = (1000000 * life.term_insurance(x, t=20, discrete=False)
            + 500000 * life.deferred_insurance(x, u=20, discrete=False))
P = benefits / payments
print(P)

SOA Question 6.27:  (D) 10310
10309.617799001708


__SOA Question 5.4__

(40) wins the SOA lottery and will receive both:
- A deferred life annuity of K per year, payable continuously, starting at age
$40 + \overset{\circ}{e}_{40}$ and
- An annuity certain of K per year, payable continuously, for $\overset{\circ}{e}_{40}$ years

You are given:
- $\mu = 0.02$
- $\delta = 0.01$
- The actuarial present value of the payments is 10,000

Calculate K.

In [7]:
print("SOA Question 5.4:  (A) 213.7")
life = ConstantForce(mu=0.02).set_interest(delta=0.01)
P = 10000 / life.certain_life_annuity(40, u=life.e_x(40, curtate=False),
                                        discrete=False)
print(P)

SOA Question 5.4:  (A) 213.7
213.74552118275955


__SOA Question 5.1__

You are given:
- $\delta_t = 0.06, \quad t \ge 0$
- $\mu_x(t) = 0.01, \quad t \ge 0$
- $Y$ is the present value random variable for a continuous annuity of 1 per year,
payable for the lifetime of (x) with 10 years certain

Calculate $Pr( Y > E[Y])$.

In [8]:
print("SOA Question 5.1: (A) 0.705")
life = ConstantForce(mu=0.01).set_interest(delta=0.06)
EY = life.certain_life_annuity(0, u=10, discrete=False)
print(life.p_x(0, t=life.Y_to_t(EY)))  # 0.705                                    


SOA Question 5.1: (A) 0.705
0.7053680433746505
